**Constantes (input)**

In [93]:
import math

# Número pi
pi_number = math.pi

# Modulo de Elasticidade
young_modulus_steel = 207 * (10**9)  # N/m²

# Coeficiente de expansão térmica
temperature_coefficient = 0.00017

# Coeficiente de Poisson
poisson_coefficient_steel = 0.3

# Constante Gravitacional
gravity = 9.81 # m/s²

**Densidades (input)**

In [94]:
# Densidade
specific_mass_steel = 7850 # Kg/m³
specific_mass_water = 1027 # Kg/m³

**Dimensões do duto (input)**

In [95]:
# Diametro
diameter = 0.40 # metros

#Espessura do aço
steel_thickness = 0.02 # metros

#Espessura do concreto
concrete_thickness = 0 # metros

# Espessura do revestimento
coating_thickness = 0 # metros

# Diâmetro Interno
diameter_intern = diameter - 2*steel_thickness

# Diametro Externo
diameter_outer = diameter + concrete_thickness + coating_thickness

**Carregamentos Funcionais (input)**

In [96]:
# Tensão Residual Efetiva
heff = 90000 # N

# Pressão interna no duto
p = 0 # bar

# Mudança de temperatura em relação à temperatura ambiente durante a instalação
delta_t = 0

**Dados do Vão Livre (input)**

In [97]:
# Comprimento da tubulação (em metros)
length = 40 # metros

# Profundidade da água
Water_depth = 110 # metros

# Distância entre o duto e o leito marinho (gap)
e_gap = 0.4 # metros


**Dados de Revestimento (input)**


In [98]:
# Fator de rigidez do concreto
kc = 0.25

# Resistência normalizada de compressão
fcn = 42 # Mpa

**Massa Efetiva (valores auxiliares)**

In [109]:

# Cálculo da massa efetiva
area_steel_pipe = pi_number * (((diameter_outer/2)**2) - ((diameter_intern/2)**2))
mass_steel_pipe =  specific_mass_steel * area_steel_pipe  # mass of the pipe

area_water = pi_number * ((diameter_outer/2) ** 2)
mass_water_displaced = specific_mass_water * area_water  #  mass of the displaced water


# Cálculo do coeficiente
if e_gap/diameter < 0.8:
  coefficient_mass_added = 0.68 + (1.6/(1+5*(e_gap/diameter)))

else:
  coefficient_mass_added = 1

mass_added = coefficient_mass_added * mass_water_displaced # additional mass

mass_fluid = 0 # mass of fluid

mass_effetive =  mass_steel_pipe + mass_added + mass_fluid

print('massa efetiva (kg/m): ' + str(mass_effetive))

massa efetiva (kg/m): 316.4840439226358


**Rigidez (valores auxiliares)**

In [110]:
# Rigidez à flexão do aço (EI_steel), dada em N.m²
moment_inertia = (pi_number/4) * (((diameter/2) ** 4) - (((diameter/2) - steel_thickness)**4))
ei_steel = moment_inertia * young_modulus_steel

print('rigidez do aço (N.m²) : ' + str(ei_steel))

rigidez do aço (N.m²) : 89456599.48355705


**Contribuição adicional para a rigidez (valores auxiliares)**

In [112]:
# Módulo de Young do Concreto
young_modulus_concrete = 10000 * (fcn ** 0.3)

ei_concrete =  moment_inertia * young_modulus_concrete

# Verificando se existe revestimento de concreto para cálculo do CSF
if concrete_thickness == 0:
  csf = 0

else:
  # Contribuição da rigidez à flexão do concreto e revestimento expressa como porcentagem de EI aço
  csf = kc * ((ei_concrete/ei_steel) ** 0.75)

print('Contribuição da rigidez à flexão do concreto e revestimento expressa como porcentagem de EI aço :' + str(csf))

Contribuição da rigidez à flexão do concreto e revestimento expressa como porcentagem de EI aço :0


**Força Axial Efetiva (valores auxiliares)**

In [102]:
import math

# Tensão axial aplicada durante a instalação da tubulação (N)
effective_lay_tension = heff

# Diferença de pressão interna (assumida como zero neste caso)
internal_pressure_diff = p

# Diferença de temperatura entre a instalação e a operação (assumida como zero neste caso)
temperature_diff = delta_t

# Cálculo da área da seção transversal da tubulação (m²)
cross_sectional_area = pi_number * (diameter ** 2) / 4

# **Cálculo da força axial efetiva**
effective_axial_force = (effective_lay_tension -  # Tensão axial de instalação
                        internal_pressure_diff * cross_sectional_area *  # Termo da pressão interna
                        (1 - 2 * poisson_coefficient_steel) -  # Ajuste devido ao efeito de Poisson
                        cross_sectional_area * young_modulus_steel * temperature_diff)  # Termo da variação térmica

print('Força Axial Efetiva: ' + str(effective_axial_force))



Força Axial Efetiva: 90000.0


**Parâmetros do solo (Clay Very Soft)**

In [103]:

# Effetive Lenght
ms = mass_steel_pipe
m = mass_water_displaced

poisson_coefficient_soil = 0.45

ds_per_d = ms/m

cv = 600000 # boundary condition coefficient (vertical dynamic stiffness)
cl = 500000 # boundary condition coefficient (lateral dynamic stiffness)

# Definição dos parâmetros de rigidez do solo para argila muito mole (Clay Very Soft)
kvs = 75000 # N/m/m - Rigidez estática vertical do solo por unidade de comprimento
kv = (cv/(1-poisson_coefficient_soil))*((2/3)*(ds_per_d) + 1/3)*math.sqrt(diameter_outer) # N/m/m - Rigidez dinâmica vertical do solo por unidade de comprimento
kl = (cl*(1+poisson_coefficient_soil))*((2/3)*(ds_per_d) + 1/3)*math.sqrt(diameter_outer) # N/m/m - Rigidez dinâmica lateral do solo por unidade de comprimento

print('Rigidez dinâmica vertical do solo: ' + str(kv))
print('Rigidez dinâmica lateral do solo: ' + str(kl))

Rigidez dinâmica vertical do solo: 897989.4426679794
Rigidez dinâmica lateral do solo: 596788.817106428


**Carga Crítica de Flambagem**




In [104]:
import math
def effetive_length(k: int, length: int, csf: int, stiffness: float):

  if k == kv:
    print('Rigidez dinâmica vertical do solo')

  elif k == kl:
    print('Rigidez dinâmica lateral do solo')

  else:
    print('Rigidez estática vertical do solo')

  # Cálculo do logaritmo do valor adimensional que influencia o comprimento efetivo
  value_log = k * (length ** 4) / ((1 + csf) * stiffness)
  print('value_log: ' + str (value_log))

  beta = math.log10(value_log)  # Cálculo do logaritmo de base 10 do valor obtido
  print('beta: ' + str(beta))  # Exibe o valor de beta

  # Cálculo da razão entre o comprimento efetivo e o comprimento total da tubulação
  ratio_length_eff = 4.73 / ((-0.066 * (beta ** 2)) + (1.02 * beta) + 0.63)

  # Cálculo do comprimento efetivo da tubulação
  length_eff = ratio_length_eff * length

  print('Comprimento Efetivo: ' + str(length_eff))  # Exibe o comprimento efetivo da tubulação

  # Coeficiente de condição de contorno
  c2 = 4

  # Cálculo da carga crítica de flambagem (Pcr)
  pcr = (1 + csf) * c2 * (pi_number ** 2) * stiffness / (length_eff ** 2)

  print('Carga Crítica de Flambagem: ' + str(pcr))  # Exibe o valor da carga crítica de flambagem

  # Razão entre uma carga específica (90000 N) e a carga crítica de flambagem
  print('Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: ' + str(effective_axial_force / pcr))  # Exibe a relação entre a carga axial efetiva e a carga crítica de flambagem
  print('')

  return length_eff, pcr

length_eff_vertical_dynamic, pcr_vertical_dynamic = effetive_length(k=kv, length=length, csf=csf, stiffness=ei_steel)
length_eff_lateral_dynamic, pcr_lateral_dynamic = effetive_length(k=kl, length=length, csf=csf, stiffness=ei_steel)
length_eff_vertical_static, pcr_vertical_static = effetive_length(k=kvs, length=length, csf=csf, stiffness=ei_steel)


Rigidez dinâmica vertical do solo
value_log: 25697.969590858167
beta: 4.409898810867445
Comprimento Efetivo: 49.21212364625961
Carga Crítica de Flambagem: 1458236.3135555368
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.06171839170604522

Rigidez dinâmica lateral do solo
value_log: 17078.442290591156
beta: 4.232448256549039
Comprimento Efetivo: 50.25501474871205
Carga Crítica de Flambagem: 1398341.7112712332
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.06436195049790866

Rigidez estática vertical do solo
value_log: 2146.2921808836627
beta: 3.3316888434025222
Comprimento Efetivo: 57.40791693322816
Carga Crítica de Flambagem: 1071589.7350489332
Razão entre Força Axial Efetiva e Carga Crítica de Flambagem: 0.08398736667246093



**Deflexão Estática**

In [105]:
# Coeficiente c2 de condição de contorno
c6 = 1/384

# Peso do tubo por unidade de comprimento
pipe_weight =  specific_mass_steel * gravity * area_steel_pipe

# Cálculo do Empuxo
empuxo = specific_mass_water * gravity * area_water

# Solicitacao Axial Efetiva
seff = effective_axial_force

# Peso Submerso por Unidade de Comprimento
q = pipe_weight - empuxo

# Cálculo da Deflexão Estática
deflection = c6 * ((q * (length_eff_vertical_static**4))/(ei_steel * (1 + csf))) * (1 / (1 + seff/pcr_vertical_static))
deflection_vertical_dynamic = c6 * ((q * (length_eff_vertical_dynamic**4))/(ei_steel * (1 + csf))) * (1 / (1 + seff/pcr_vertical_dynamic))

print('Deflexão: ' + str(deflection))
print('Razão entre deflexão e diâmetro: '+ str(deflection/diameter))

Deflexão: 0.16702593585513886
Razão entre deflexão e diâmetro: 0.41756483963784713


**Amplitude máxima de tensão**

In [106]:
# Coeficiente C4 de condição de contorno
length_eff_list = [length_eff_vertical_dynamic, length_eff_lateral_dynamic, length_eff_vertical_static]

for length_eff in length_eff_list:

  c4_midspan = 8.6
  c4_shouder = 14.1 * ((length/length_eff)**2)
  c4 = [c4_midspan, c4_shouder]

  if length_eff == length_eff_vertical_dynamic:
    name_length_eff = 'Rigidez dinâmica vertical do solo'

  elif length_eff == length_eff_lateral_dynamic:
    name_length_eff = 'Rigidez dinâmica lateral do solo'

  else:
     name_length_eff ='Rigidez estática vertical do solo'

  for c in c4:

    if c == c4_midspan:
      name_c = 'midspan'

    else:
      name_c = 'shouder'

    radius_extern = diameter / 2
    radius_intern = (diameter - 1*steel_thickness)/2
    radius_list = [radius_extern, radius_intern]

    for radius in radius_list:

      if radius == radius_extern:
        print(name_length_eff + ', ' + name_c + ', ' +' raio externo')

      else:
        print(name_length_eff + ', ' + name_c +', ' + 'raio interno')

      amplitude_max_inline = 2 * c * (1 + csf) * diameter * young_modulus_steel * radius / (length_eff ** 2)
      #amplitude_max_crossflow = 2 * c * (1 + csf) * diameter * young_modulus * radius / (length_eff ** 2)

      print('Amplitude (MPa): ' + str(amplitude_max_inline/1000000))
      #print('Amplitude cross-flow (MPa): ' + str(amplitude_max_crossflow/1000000))
      print('')

Rigidez dinâmica vertical do solo, midspan,  raio externo
Amplitude (MPa): 117.61008567433946

Rigidez dinâmica vertical do solo, midspan, raio interno
Amplitude (MPa): 111.72958139062249

Rigidez dinâmica vertical do solo, shouder,  raio externo
Amplitude (MPa): 127.39166017261526

Rigidez dinâmica vertical do solo, shouder, raio interno
Amplitude (MPa): 121.02207716398449

Rigidez dinâmica lateral do solo, midspan,  raio externo
Amplitude (MPa): 112.77944934975642

Rigidez dinâmica lateral do solo, midspan, raio interno
Amplitude (MPa): 107.14047688226859

Rigidez dinâmica lateral do solo, shouder,  raio externo
Amplitude (MPa): 117.14177604714081

Rigidez dinâmica lateral do solo, shouder, raio interno
Amplitude (MPa): 111.28468724478375

Rigidez estática vertical do solo, midspan,  raio externo
Amplitude (MPa): 86.42615697832706

Rigidez estática vertical do solo, midspan, raio interno
Amplitude (MPa): 82.1048491294107

Rigidez estática vertical do solo, shouder,  raio externo
Ampl

In [107]:
amplitude_tensao_crossflow = (127.391 + 114.652)/2
print('amplitude_tensao_crossflow: '+ str(amplitude_tensao_crossflow))

amplitude_tensao_inline = (117.141 + 105.427)/2
print('amplitude_tensao_inline: '+ str(amplitude_tensao_inline))

amplitude_tensao_crossflow: 121.0215
amplitude_tensao_inline: 111.284


**Frequência Natural Fundamental**

In [108]:
# Boundary condition coefficients
c1_inline, c1_crossflow = 3.56, 3.56
c3_inline, c3_crossflow = 0, 0.4

# Massa efetiva
me = mass_effetive

# Cálculo das Frequências
f_inline = c1_inline * math.sqrt(1 + csf) * math.sqrt(ei_steel/(me * (length_eff_lateral_dynamic ** 4)) * (1 + (seff/pcr_lateral_dynamic) + c3_inline * (deflection/diameter) ** 2))
f_crossflow = c1_crossflow * math.sqrt(1 + csf) * math.sqrt(ei_steel/(me * (length_eff_vertical_dynamic ** 4)) * (1 + (seff/pcr_vertical_dynamic) + c3_crossflow * (deflection/diameter) ** 2))

print('Frequência na direção in-line (Hz) :'+ str(f_inline))
print('Frequência na direção cross-flow (Hz) :'+ str(f_crossflow))

Frequência na direção in-line (Hz) :0.773154239826187
Frequência na direção cross-flow (Hz) :0.8312968979996561
